In [ ]:
# Import Packages 

import tensorflow as tf
import keras
import json
from PIL import Image
import os
import pandas as pd
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from collections import defaultdict
import shutil
import random

### Processing Data

In [ ]:

annotations_file = 'path\to\annotations'
with open(annotations_file, 'r') as file:
    annotations = json.load(file)

# Parse annotations
data_directory = r'\path\to\directory'
data = []
for filename, file_data in annotations.items():

    image_path = data_directory+'\\'+filename.split('.')[0]+'.png'
    image = Image.open(image_path)
    for region in file_data['regions']:

        shape_attrs = region['shape_attributes']
        region_attrs = region['region_attributes']
        temp = {
            'filename': filename.split('.')[0]+'.png',
            'name': shape_attrs['name'],
            'width': image.width,
            'height': image.height,
            'all_points_x': shape_attrs['all_points_x'],
            'all_points_y': shape_attrs['all_points_y'],
            'Type': region_attrs['Type']
        }

        #Don't include duplicates
        if temp not in data:
            data.append({
                'filename': filename.split('.')[0]+'.png',
                'name': shape_attrs['name'],
                'width': image.width,
                'height': image.height,
                'all_points_x': shape_attrs['all_points_x'],
                'all_points_y': shape_attrs['all_points_y'],
                'Type': region_attrs['Type']
            })

In [ ]:
def convert(size, box):
    dw = 1. / size[0]
    dh = 1. / size[1]
    x = (box[0] + box[1]) / 2.0
    y = (box[2] + box[3]) / 2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return (x, y, w, h)

In [ ]:
def convert_polygon_to_bbox(x_coords,y_coords):
    xmin = min(x_coords)
    xmax = max(x_coords)
    ymin = min(y_coords)
    ymax = max(y_coords)
    return [xmin, xmax, ymin, ymax]

In [ ]:
def convert_annotations_to_yolo(data_list, output_dir, label_map):
    annotations_by_filename = defaultdict(list)
    
    for data in data_list:
        filename = data['filename']
        width = data['width']
        height = data['height']
        all_points_x = data['all_points_x']
        all_points_y = data['all_points_y']
        label = data['Type']
        
        bbox = convert_polygon_to_bbox(all_points_x, all_points_y)
        bb = convert((width, height), bbox)
        class_id = label_map[label]
        
        annotations_by_filename[filename].append((class_id, bb))
    
    # Write annotations to YOLO format files
    for filename, annotations in annotations_by_filename.items():
        file_base = os.path.splitext(filename)[0]
        with open(f'{output_dir}/{file_base}.txt', 'w') as out_file:
            for class_id, bb in annotations:
                out_file.write(f"{class_id} {' '.join(map(str, bb))}\n")



label_map = {'Offense': 0, 'Defense': 1}  
output_dir =r'path\to\output'
os.makedirs(output_dir, exist_ok=True)


convert_annotations_to_yolo(data, output_dir, label_map)

In [ ]:
def create_dirs(base_dir, categories):
    for category in categories:
        os.makedirs(os.path.join(base_dir, category, 'images'), exist_ok=True)
        os.makedirs(os.path.join(base_dir, category, 'labels'), exist_ok=True)

def split_dataset(image_dir, annotation_dir, train_dir, test_dir, split_ratio=0.8):
    images = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))]
    annotations = [f for f in os.listdir(annotation_dir) if os.path.isfile(os.path.join(annotation_dir, f))]


    if len(images) != len(annotations):




        print("Number of images and annotations do not match!")
        return

    paired_files = list(zip(images, annotations))
    random.shuffle(paired_files)
    
    train_size = int(len(paired_files) * split_ratio)
    train_files = paired_files[:train_size]
    test_files = paired_files[train_size:]
    
    for files in train_files:
        shutil.copy(os.path.join(image_dir, files[0]), os.path.join(train_dir, 'images', files[0]))
        shutil.copy(os.path.join(annotation_dir, files[1]), os.path.join(train_dir, 'labels', files[1]))
    
    for files in test_files:
        shutil.copy(os.path.join(image_dir, files[0]), os.path.join(test_dir, 'images', files[0]))
        shutil.copy(os.path.join(annotation_dir, files[1]), os.path.join(test_dir, 'labels', files[1]))

if __name__ == "__main__":
    base_dir = r'path\to\directory'
    image_dir = os.path.join(base_dir, 'NFL All 22 Formations Data_Yolo_Images')
    annotation_dir = os.path.join(base_dir, 'YOLO_labels')
    train_dir = os.path.join(base_dir, 'train')
    test_dir = os.path.join(base_dir, 'test')
    
    create_dirs(base_dir, ['train', 'test'])
    split_dataset(image_dir, annotation_dir, train_dir, test_dir, split_ratio=0.8)

In [1]:
import yaml

data = {
    'train': r'path\to\train',
    'val': r'path\to\val',
    'nc': 2,
    'names': [
 'Offense', 'Defense']
}

with open('model.yaml', 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

### Training Model

In [ ]:
model = YOLO("yolov8m.pt")

In [ ]:
#Adjust parameters if needed
model.train(data='path\to\model.yaml', epochs=100, imgsz=640, batch=4,patience =7)

In [ ]:
metrics = model.val(data='path\to\model.yaml', imgsz=640)

In [ ]:
model.export()